In [75]:
from PIL import Image
import math
import numpy as np

"""
mapped_sum_gradient_iguanaet and use the functions associated with gaussconvolve2d that you used in the last HW02.
"""
def gauss1d(sigma):
    width: int = int(math.ceil(6 * sigma))  # 6*sigma의 다음 정수로 width를 설정
    if width % 2 == 0:  # 다음 정수가 짝수라면 +1 하여 홀수로 변경
        width += 1

    center = math.floor(width / 2)  # 가운데 index를 획득
    gaussian = np.zeros((1, width)).flatten()  # 반환할 1D Numpy Array 생성
    for i in range(width):
        gaussian[i] = i - center  # Hint:를 토대로 x값을 입력
    for i in range(width):
        gaussian[i] = math.exp(-gaussian[i] ** 2 / (2 * sigma ** 2))  # x값에 따라 gaussian값 획득 및 입력

    # 아래는 normalize, 전체 합으로 각 엔트리를 나눈다.
    kernal_sum = np.sum(gaussian)
    for i in range(width):
        gaussian[i] /= kernal_sum
    return gaussian

def gauss2d(sigma):
    return np.outer(gauss1d(sigma), gauss1d(sigma))
# outer product로 2d 가우시안 필터를 구한다.

def convolve2d(array, filter):
    padding_width = int(len(filter) / 2)  # filter에 따른 padding 크기
    zero_padding = np.zeros((np.shape(array)[0] + padding_width * 2,
                             np.shape(array)[1] + padding_width * 2))
    # padding한 크기에 맞게 새로운 np.array를 생성

    # convolution연산을 위해서 filter를 축의 방향에 대해서 뒤집는다.
    flipped_filter = np.flip(filter, axis=0)
    flipped_filter = np.flip(flipped_filter, axis=1)
    # 모든 픽셀에 대해서 paading을 제외한 부분에 array를 집어넣어 준다.
    for i in range(padding_width, np.shape(zero_padding)[0] - padding_width):
        for j in range(padding_width, np.shape(zero_padding)[1] - padding_width):
            zero_padding[i][j] = array[i - padding_width][j - padding_width]

    # padding된 이미지의 각 픽셀을 순회하면서 각 픽셀에 컨벌루젼한 값을 입력한다.

    convolved = np.zeros(np.shape(array))

    for i in range(padding_width, np.shape(zero_padding)[0] - padding_width):
        for j in range(padding_width, np.shape(zero_padding)[1] - padding_width):
            subpart = zero_padding[i - padding_width:i + padding_width + 1,
                      j - padding_width:j + padding_width + 1]
            convolved[i-padding_width][j-padding_width] = np.sum(subpart * flipped_filter)
    # 컨벌루젼한 값을 리턴
    return convolved

def gaussconvolve2d(array, sigma):
    return convolve2d(array, gauss2d(sigma))


In [76]:
# 이구아나 사진을 불러온다.
iguana_img = Image.open('iguana.bmp')
# 사진을 흑백처리한다.
iguana_img_grey = iguana_img.convert('L')
# 이미지를 np 배열형태로 변형한다.
iguana_array_grey = np.asarray(iguana_img_grey)
# 가우시안 필터 효과를 적용한다.
iguana_array_grey_blur = np.uint8(gaussconvolve2d(iguana_array_grey, 1.6))
# 배열을 이미지로 변경하여 보여준다.
Image.fromarray(iguana_array_grey_blur).show()

In [77]:
#sobel연산을 위한 필터를 제작한다. normalization을 위해 8로 나눈다.
x_sobel = np.array([ 1, 0, -1, 2, 0, -2, 1, 0, -1])
y_sobel = np.array([ 1, 2, 1, 0, 0, 0, -1, -2, -1])
x_sobel = x_sobel.reshape((3, 3))/8.0
y_sobel = y_sobel.reshape((3, 3))/8.0

In [78]:
# sobel
x_gradient_iguana = convolve2d(iguana_array_grey_blur, x_sobel)
y_gradient_iguana = convolve2d(iguana_array_grey_blur, y_sobel)

In [79]:
sum_gradient_iguana = np.hypot(x_gradient_iguana, y_gradient_iguana)
mapped_sum_gradient_iguana = (sum_gradient_iguana / np.max(sum_gradient_iguana)) * 255

In [80]:
Image.fromarray(mapped_sum_gradient_iguana).show()

In [81]:
vectorized_arctan = np.vectorize(np.arctan)
x_gradient_iguana[x_gradient_iguana==0] = 0.000001
theta = vectorized_arctan(y_gradient_iguana/x_gradient_iguana)

theta_degree = theta * 180 / np.pi
# 변환된 각도에 따라 매핑한다. 매핑은 0 ~
theta_mapped = (theta_degree + 22.5) // 45
print(theta_degree)
print(np.max(theta_degree))
print(np.min(theta_degree))
print("------")
print(theta_mapped)
print(np.max(theta_mapped))
print(np.min(theta_mapped))

[[ 45.          77.15229514  86.42366563 ... -84.55966797 -75.5432446
  -44.6527571 ]
 [ 15.42216132  50.19442891  75.46554492 ... -68.02549201 -43.85423716
  -13.77366153]
 [  7.65065096  29.475889    60.2551187  ... -47.91083783 -23.78203042
   -6.38282253]
 ...
 [ 13.39249775  39.09385889  54.78240703 ... -89.99999673  57.72435569
   13.70230016]
 [  3.17983012  11.88865804  21.80140949 ... -83.57652956  72.09555249
   20.58445165]
 [-39.28940686 -67.93210044 -75.37912601 ... -85.04154908  85.19204604
   48.27048792]]
89.99999889283518
-89.99999934331485
------
[[ 1.  2.  2. ... -2. -2. -1.]
 [ 0.  1.  2. ... -2. -1.  0.]
 [ 0.  1.  1. ... -1. -1.  0.]
 ...
 [ 0.  1.  1. ... -2.  1.  0.]
 [ 0.  0.  0. ... -2.  2.  0.]
 [-1. -2. -2. ... -2.  2.  1.]]
2.0
-2.0


theta 를 매핑하고 0/45/90/135
매핑된 결과에 따라서 가장 큰 값을 적용

In [82]:
def remain_max(array, x, y, angle):
    ret = 0
    if angle == -2 or angle == 2:
        if array[y][x] < max(array[y+1][x], array[y][x], array[y-1][x]):
            ret = 0
        else:
            ret =  array[y][x]
    if angle == -1:
        if array[y][x] < max(array[y+1][x-1], array[y][x], array[y-1][x+1]):
            ret =  0
        else:
            ret =  array[y][x]
    if angle == 0:
        if array[y][x] < max(array[y][x+1], array[y][x], array[y][x-1]):
            ret = 0
        else:
            ret = array[y][x]
    if angle == 1:
        if array[y][x] < max(array[y+1][x+1], array[y][x], array[y-1][x-1]):
            ret = 0
        else:
            ret = array[y][x]
    return ret

In [83]:
res = np.zeros(np.shape(mapped_sum_gradient_iguana))
for row in range(1, len(mapped_sum_gradient_iguana)-1):
    for col in range(1, len(mapped_sum_gradient_iguana[1])-1):
        res[row][col] = \
            remain_max(mapped_sum_gradient_iguana, col, row, theta_mapped[row][col])
Image.fromarray(res).show()

In [84]:
def theta_mapping(angle):
    if angle == -2 or angle == 2:
        return np.array([128,0,0])
    if angle == -1:
        return np.array([0,128,0])
    if angle == 0:
        return np.array([0,0,128])
    if angle == 1:
        return np.array([128,128,128])

In [85]:
mapped_theta = np.zeros((len(theta), len(theta[0]), 3))
for i in range(len(mapped_theta)):
    for j in range(len(mapped_theta[0])):
        for k in range(3):
            mapped_theta[i][j][k] = theta_mapping(theta_mapped[i][j])[k]

print(mapped_theta)
Image.fromarray(mapped_theta.astype(np.uint8)).show()

[[[128. 128. 128.]
  [128.   0.   0.]
  [128.   0.   0.]
  ...
  [128.   0.   0.]
  [128.   0.   0.]
  [  0. 128.   0.]]

 [[  0.   0. 128.]
  [128. 128. 128.]
  [128.   0.   0.]
  ...
  [128.   0.   0.]
  [  0. 128.   0.]
  [  0.   0. 128.]]

 [[  0.   0. 128.]
  [128. 128. 128.]
  [128. 128. 128.]
  ...
  [  0. 128.   0.]
  [  0. 128.   0.]
  [  0.   0. 128.]]

 ...

 [[  0.   0. 128.]
  [128. 128. 128.]
  [128. 128. 128.]
  ...
  [128.   0.   0.]
  [128. 128. 128.]
  [  0.   0. 128.]]

 [[  0.   0. 128.]
  [  0.   0. 128.]
  [  0.   0. 128.]
  ...
  [128.   0.   0.]
  [128.   0.   0.]
  [  0.   0. 128.]]

 [[  0. 128.   0.]
  [128.   0.   0.]
  [128.   0.   0.]
  ...
  [128.   0.   0.]
  [128.   0.   0.]
  [128. 128. 128.]]]


Theta는 제대로 작동하는 것처럼 보이는데....

In [86]:
def apply_double_threshold(entry, T_high, T_low):
    if entry >= T_high:
        return 255
    elif T_high > entry >= T_low:
        return 80
    else:
        return 0

In [87]:
def double_thresholding(img):
    diff = img.max() - img.min()
    T_high = img.min() + diff * 0.15
    T_low = img.min() + diff * 0.03

    res = img[:]

    vectorized_threshold = np.vectorize(apply_double_threshold)
    res = vectorized_threshold(img, T_high, T_low)
    """
    Args:
        img: numpy array of shape (H, W) representing NMS edge response.
    Returns:
        res: double_thresholded image.
    """
    return res

In [88]:
double_thresholded = double_thresholding(res)
Image.fromarray(double_thresholded.astype(np.uint8)).show()

In [89]:
def dfs(img, res, i, j, visited=[]):
    # 호출된 시점의 시작점 (i, j)은 최초 호출이 아닌 이상
    # strong 과 연결된 weak 포인트이므로 res에 strong 값을 준다
    res[i, j] = 255

    # 이미 방문했음을 표시한다
    visited.append((i, j))

    # (i, j)에 연결된 8가지 방향을 모두 검사하여 weak 포인트가 있다면 재귀적으로 호출
    for ii in range(i-1, i+2) :
        for jj in range(j-1, j+2) :
            if (img[ii, jj] == 80) and ((ii, jj) not in visited) :
                dfs(img, res, ii, jj, visited)

def hysteresis(img):
    res = np.zeros(np.shape(img))
    visited = []
    width = len(img)
    height = len(img[0])
    for i in range(1, width-1):
        for j in range(1, height-1):
            if img[i][j] == 255:
                dfs(img, res, i, j, visited)
    """ Find weak edges connected to strong edges and link them.
    Iterate over each pixel in strong_edges and perform depth first
    search across the connected pixels in weak_edges to link them.
    Here we consider a pixel (a, b) is connected to a pixel (c, d)
    if (a, b) is one of the eight neighboring pixels of (c, d).
    Args:
        img: numpy array of shape (H, W) representing NMS edge response.
    Returns:
        res: hysteresised image.
    """
    return res

In [90]:
hysteresised = hysteresis(double_thresholded)
Image.fromarray(hysteresised.astype(np.uint8)).show()